### 목적
- LangGraph의 런타임 객체 정보 액세스

#### 런타임(Runtime) 정의
- 그래프가 실행되는 컨텍스트(상태 + 도구 + 의존성)을 다루는 핵심 엔진
- LangGraph/LangChain은 내부적으로 모든 Agent를 "그래프"처럼 실행시킨다.
- 이 때 각 노드(모델 호출, 툴 호출, 미들웨어 등)가 돌아가는 환경 전체를 **Runtime**이 관리한다.

#### 런타임이 하는 일
| 구성요소              | 역할                                             | 비유                      |
| ----------------- | ---------------------------------------------- | ----------------------- |
| **Context**       | 실행 중 필요한 외부 의존성 (user_id, DB 연결, API client 등) | “컨테이너 안으로 들어온 환경 변수 세트” |
| **Store**         | 그래프 밖에서도 유지되는 장기 저장소 (BaseStore)               | “하드디스크”                 |
| **Stream Writer** | 스트리밍 이벤트(custom updates)를 프론트/UI로 흘려보냄         | “실시간 방송 송출기”            |


#### 툴 내부
- 툴 내에서 런타임 정보에 접근하여 아래 기능을 수행한다.
    - 컨텍스트 접근
    - 장기 메모리를 읽거나 쓰기
    - 사용자 정의 스트림 출력

In [60]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from typing_extensions import TypedDict
from langchain.tools import tool, ToolRuntime
# ToolRuntime: 툴 함수 안에서 에이전트 실행환경(runtime)에 바로 접근하게 해주는 주입 파라미터
from langgraph.store.memory import InMemoryStore
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

class Context(TypedDict):
    user_id: str

@tool
def fetch_user_email_preferences(runtime: ToolRuntime[Context]) -> str:
    # 툴에 runtime 파라미터를 받겠다고 명시하면, 에이전트가 해당 툴을 실행할 때 프레임워크가 자동으로 주입한다.
    ## -> 파라미터 명은 상관없지만 타입 힌트는 반드시 ToolRuntime이어야 한다.
    ## -> 런타임 사용하지 않는다면 안 받아도 된다.
    ## -> LLM이 해당 툴을 사용하겠다고 판단했을 때, 인자가 없는 것으로 보인다.(LLM이 넣는 것이 아닌 내부 DI가 넣기 때문이다.)
    """저장소에서 사용자의 이메일 선호를 가져옵니다."""
    user_id = runtime.context['user_id']

    # 기본 선호 문구
    preferences: str = "사용자는 간결하고 정중한 이메일을 작성하는 것을 선호합니다." 

    if runtime.store:
        if memory := runtime.store.get(('users',), user_id): # :=는 값을 할당하면서 바로 쓰는 "왈러스 연산자"이다.
            preferences = memory.value['preferences']

    return preferences

# 메모리 준비 -> 사전 데이터 적재
store = InMemoryStore()
store.put(("users",), "test", {"preferences": "명확성 + 구체성"})

agent = create_agent(
    model=llm,
    tools=[fetch_user_email_preferences],
    store=store, # 실제 store 연결
    context_schema=Context
)

result = agent.invoke(
    {"messages":[{'role': 'user', 'content': '사용자의 이메일 선호 보여줘'}]}, 
    context=Context(user_id='test')
)

print(result.get('messages')[-1].content)

사용자의 이메일 선호는 "명확성 + 구체성"입니다.


#### 미들웨어 내부
- 미들웨어에서 런타임 정보에 접근하여 에이전트 동작을 제어할 수 있다. (동적 프롬프트 생성, 메시지 수정 등)

In [62]:
from dataclasses import dataclass

from langchain.messages import AnyMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
from langgraph.runtime import Runtime

load_dotenv()

llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))

@dataclass
class Context:
    user_name: str

@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    user_name = request.runtime.context.user_name
    return f"너는 유용한 어시스턴트이다. 유저의 이름은 {user_name}이다."

@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:
    print(f"Completed request for user: {runtime.context.user_name}")  
    return None

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[dynamic_system_prompt, log_before_model, log_after_model],
    context_schema=Context,
)

agent.invoke(
    {"messages": [{"role": "user", "content": "나의 이름은 뭐야?"}]},
    context=Context(user_name="홍길동")  
)

Processing request for user: 홍길동
Completed request for user: 홍길동


{'messages': [HumanMessage(content='나의 이름은 뭐야?', additional_kwargs={}, response_metadata={}, id='a0554ae4-1c29-45a7-8597-4f4d3f7dd3fc'),
  AIMessage(content='당신의 이름은 홍길동이에요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 40, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYp2l0a6m9FZ1wBPWG664ztqIyyRZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--6926aaa6-c6b9-465d-bb74-ec7cbe7d2575-0', usage_metadata={'input_tokens': 40, 'output_tokens': 11, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}